In [19]:
# # Generate Random Users Data

# Tạo 20 users ngẫu nhiên và insert vào database

# ## Schema Table Users

# ```sql
# create table public.users (
#   user_id serial not null,
#   username character varying(50) not null,
#   full_name character varying(50) not null,
#   email character varying(50) not null,
#   password character varying(100) not null,
#   address character varying(100) null,
#   role character varying(10) null default 'user'::character varying,
#   date_of_birth date null,
#   rating double precision null default 0.0,
#   rating_count integer null default 0,
#   created_at timestamp with time zone null default now(),
#   seller_end_at timestamp with time zone null,
#   constraint users_pkey primary key (user_id),
#   constraint users_email_key unique (email)
# ) TABLESPACE pg_default;
# ```

In [20]:
%pip install Faker bcrypt

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import json
import random
import bcrypt
import psycopg2
from datetime import datetime, timedelta
from dotenv import load_dotenv
from faker import Faker

load_dotenv()

SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")
fake = Faker('vi_VN')  # Sử dụng Faker tiếng Việt
SALT_ROUNDS = 10

print("✅ Đã import tất cả thư viện")

✅ Đã import tất cả thư viện


## 1. Helper Functions

In [22]:
def hash_password(password: str) -> str:
    """Hash password using bcrypt (tương tự server)"""
    salt = bcrypt.gensalt(rounds=SALT_ROUNDS)
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed.decode('utf-8')

def generate_random_user():
    """Generate random user data"""
    full_name = fake.name()
    email = fake.email()
    username = email.split('@')[0]  # Tạo username từ email
    password = "Lok@1412"  # Password mặc định
    address = fake.address().replace('\n', ', ')
    
    return {
        'username': username,
        'full_name': full_name,
        'email': email,
        'password': password,
        'address': address
    }

def generate_random_date_of_birth():
    """Generate random date of birth (18-70 tuổi)"""
    today = datetime.now()
    start_date = today - timedelta(days=70*365)
    end_date = today - timedelta(days=18*365)
    
    random_days = random.randint(0, (end_date - start_date).days)
    return (start_date + timedelta(days=random_days)).date()

def generate_random_seller_end_at():
    """Generate random seller_end_at hoặc None"""
    if random.random() < 0.3:  # 30% chance có seller_end_at
        today = datetime.now()
        # Random từ 1 tháng trước đến 6 tháng sau
        days_offset = random.randint(-30, 180)
        return today + timedelta(days=days_offset)
    return None

print("✅ Đã định nghĩa các hàm helper")

✅ Đã định nghĩa các hàm helper


In [23]:
# Generate và insert 20 users trực tiếp vào database
NUM_USERS = 20

created_users = []
success_count = 0
error_count = 0

print(f"🚀 Bắt đầu tạo {NUM_USERS} users...\n")

try:
    # Kết nối Supabase
    print("⚙️ Đang kết nối Supabase...")
    if SUPABASE_CONNECTION_STRING:
        conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
        cursor = conn.cursor()
        print("✅ Kết nối Supabase thành công!\n")
    else:
        print("❌ Không tìm thấy SUPABASE_CONNECTION_STRING trong .env")
        raise Exception("Missing SUPABASE_CONNECTION_STRING")
    
    for i in range(NUM_USERS):
        user_data = generate_random_user()
        
        try:
            # Hash password
            hashed_password = hash_password(user_data['password'])
            
            # Random date_of_birth và seller_end_at
            date_of_birth = generate_random_date_of_birth()
            seller_end_at = generate_random_seller_end_at()
            
            # Insert vào database
            cursor.execute("""
                INSERT INTO users (username, full_name, email, password, address, date_of_birth, seller_end_at)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                RETURNING user_id, email, full_name
            """, (
                user_data['username'],
                user_data['full_name'],
                user_data['email'],
                hashed_password,
                user_data['address'],
                date_of_birth,
                seller_end_at
            ))
            
            conn.commit()
            user_id, email, full_name = cursor.fetchone()
            
            success_count += 1
            created_users.append({
                'user_id': user_id,
                'email': email,
                'full_name': full_name
            })
            
            seller_info = seller_end_at.strftime('%Y-%m-%d') if seller_end_at else "None"
            print(f"✅ [{i+1}/{NUM_USERS}] Created: {full_name} ({email})")
            print(f"   DOB: {date_of_birth}, Seller_End: {seller_info}")
            
        except Exception as e:
            conn.rollback()
            error_count += 1
            print(f"❌ [{i+1}/{NUM_USERS}] Error: {user_data['email']} - {e}")
    
    # Đóng connection
    cursor.close()
    conn.close()
    
    print(f"\n📊 Kết quả:")
    print(f"   ✅ Thành công: {success_count}")
    print(f"   ❌ Thất bại: {error_count}")
    print(f"   📊 Tổng: {NUM_USERS}")
    print(f"\n🔒 Đã đóng kết nối Supabase")
    
except Exception as e:
    print(f"❌ Lỗi kết nối: {e}")
    import traceback
    traceback.print_exc()

🚀 Bắt đầu tạo 20 users...

⚙️ Đang kết nối Supabase...
✅ Kết nối Supabase thành công!

✅ [1/20] Created: Trọng Mai Bảo Hoàng (janele@example.com)
   DOB: 1977-03-11, Seller_End: None
✅ [2/20] Created: Nam Văn Bùi (wnguyen@example.com)
   DOB: 2007-03-08, Seller_End: None
✅ [3/20] Created: An Bảo Mai (enguyen@example.org)
   DOB: 1973-10-08, Seller_End: None
✅ [4/20] Created: Huy Đặng (duongjane@example.org)
   DOB: 1999-03-02, Seller_End: None
✅ [5/20] Created: Anh Châu Hoàng (janemai@example.org)
   DOB: 2002-10-30, Seller_End: 2026-01-13
✅ [6/20] Created: Lâm Mai (janevu@example.com)
   DOB: 1981-11-05, Seller_End: None
✅ [7/20] Created: Hoàng Hoàng (john80@example.org)
   DOB: 1962-04-25, Seller_End: 2025-12-05
✅ [8/20] Created: Hạnh Hữu Dương (john39@example.net)
   DOB: 1957-06-27, Seller_End: 2026-05-01
✅ [9/20] Created: Dũng Đặng (maijohn@example.com)
   DOB: 1960-03-14, Seller_End: 2026-03-17
✅ [10/20] Created: Ông Trọng Vũ (johnnguyen@example.net)
   DOB: 1987-12-27, Seller_En

## 2. Generate và Insert 20 Users trực tiếp vào Database

In [24]:
# # Xóa tất cả users vừa tạo (nếu cần)
# try:
#     print("⚙️ Đang kết nối Supabase...")
#     if SUPABASE_CONNECTION_STRING:
#         conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
#         cursor = conn.cursor()
#         print("✅ Kết nối Supabase thành công!\n")
#     else:
#         print("❌ Không tìm thấy SUPABASE_CONNECTION_STRING trong .env")
#         raise Exception("Missing SUPABASE_CONNECTION_STRING")
    
#     if 'created_users' in locals() and created_users:
#         user_ids = [user['user_id'] for user in created_users]
#         placeholders = ','.join(['%s'] * len(user_ids))
        
#         cursor.execute(f"""
#             DELETE FROM users
#             WHERE user_id IN ({placeholders})
#         """, user_ids)
        
#         conn.commit()
#         deleted_count = cursor.rowcount
        
#         print(f"✅ Đã xóa {deleted_count} users")
#         for user in created_users:
#             print(f"   - {user['full_name']} ({user['email']})")
        
#         # Clear danh sách
#         created_users.clear()
#     else:
#         print("⚠️ Không có users nào để xóa (biến created_users trống)")
    
#     cursor.close()
#     conn.close()
#     print("\n🔒 Đã đóng kết nối Supabase")
    
# except Exception as e:
#     print(f"❌ Lỗi: {e}")
#     import traceback
#     traceback.print_exc()

## 3. [Optional] Xóa Users vừa tạo